In [8]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np

In [9]:
def distance_mahalanobis(instance1, instance2, k=None, eps=1e-6):
    """
    單向 Mahalanobis distance
    """
    mean1, cov1, _ = instance1
    mean2, _, _    = instance2

    diff = mean1 - mean2
    dim = cov1.shape[0]
    cov1_reg = cov1 + eps * np.eye(dim)
    inv_cov1 = np.linalg.inv(cov1_reg)
    return diff.T @ inv_cov1 @ diff

def distance_kl(instance1, instance2, k=None):
    """
    Kullback-Leibler Divergence
    """
    mm1, cm1, _ = instance1
    mm2, cm2, _ = instance2
    dim = cm1.shape[0]
    
    term1 = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    diff = mm2 - mm1
    term2 = np.dot(np.dot(diff.transpose(), np.linalg.inv(cm2)), diff)
    term3 = np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    
    return term1 + term2 + term3 - dim

def getNeighbors(trainingSet, instance, k, distance_func=distance_mahalanobis):
    distances = []
    for x in range(len(trainingSet)):
        dist = distance_func(trainingSet[x], instance, k) + distance_func(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

def nearestClass(neighbors):
    classVote = {}
    for response in neighbors:
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1
    sorter = sorted(classVote.items(), key=operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [10]:
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)

In [11]:
directory = "genres/"
f= open("my.dat" ,'wb')
i=0
for folder in os.listdir(directory):
    if not os.path.isdir(os.path.join(directory, folder)):
        continue
    i+=1
    if i==11 :
        break   
    for file in os.listdir(directory+folder):  
        (rate,sig) = wav.read(directory+folder+"/"+file)
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix , covariance , i)
        pickle.dump(feature , f)
        print(f"File: {folder}/{file}")
        print(f"Genre index: {i}")
        print(f"MFCC Features shape: {mfcc_feat.shape}")
        print(f"Mean Matrix: {mean_matrix}")
        print(f"Covariance Matrix shape: {covariance.shape}")
        print(f"Feature tuple: (mean_matrix, covariance, genre_index={i})")
        print("-" * 80)
f.close()

File: blues/blues.00000.wav
Genre index: 1
MFCC Features shape: (2994, 13)
Mean Matrix: [ 76.50261107  -1.96141736 -15.7743469    3.8314189  -10.47332553
   1.31182833 -19.39373183   5.28678994 -16.63172467   5.3534445
  -9.85657026   6.25007663  -5.58749505]
Covariance Matrix shape: (13, 13)
Feature tuple: (mean_matrix, covariance, genre_index=1)
--------------------------------------------------------------------------------
File: blues/blues.00001.wav
Genre index: 1
MFCC Features shape: (2994, 13)
Mean Matrix: [ 66.93164965   0.71885345  -3.26783036   4.18120038  -8.05094092
   6.4482592  -17.76851742  14.09181008 -18.33253624   3.68556046
 -10.55074362   2.88748511  -7.78808286]
Covariance Matrix shape: (13, 13)
Feature tuple: (mean_matrix, covariance, genre_index=1)
--------------------------------------------------------------------------------
File: blues/blues.00002.wav
Genre index: 1
MFCC Features shape: (2994, 13)
Mean Matrix: [ 78.77410313   3.03970009 -19.02443546  -0.98359

In [12]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break  

    for x in range(len(dataset)):
        if random.random() <split :      
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])  

trainingSet = []
testSet = []
loadDataset("my.dat" , 0.66, trainingSet, testSet)

In [13]:
leng = len(testSet)

print("--- Mahalanobis Distance ---")
predictions_mah = []
for x in range(leng):
    # print(f"Classifying test sample {x+1}/{leng}")
    predictions_mah.append(nearestClass(getNeighbors(trainingSet, testSet[x], 5, distance_func=distance_mahalanobis)))

accuracy_mah = getAccuracy(testSet, predictions_mah)
print("Accuracy (Mahalanobis):", accuracy_mah)

print("\n--- KL Divergence ---")
predictions_kl = []
for x in range(leng):
    # print(f"Classifying test sample {x+1}/{leng}")
    predictions_kl.append(nearestClass(getNeighbors(trainingSet, testSet[x], 5, distance_func=distance_kl)))

accuracy_kl = getAccuracy(testSet, predictions_kl)
print("Accuracy (KL Divergence):", accuracy_kl)

Classifying test sample 1/326
Prediction: 9
Classifying test sample 2/326
Prediction: 1
Classifying test sample 3/326
Prediction: 3
Classifying test sample 4/326
Prediction: 10
Classifying test sample 5/326
Prediction: 6
Classifying test sample 6/326
Prediction: 1
Classifying test sample 7/326
Prediction: 1
Classifying test sample 8/326
Prediction: 1
Classifying test sample 9/326
Prediction: 1
Classifying test sample 10/326
Prediction: 1
Classifying test sample 11/326
Prediction: 1
Classifying test sample 12/326
Prediction: 1
Classifying test sample 13/326
Prediction: 1
Classifying test sample 14/326
Prediction: 1
Classifying test sample 15/326
Prediction: 1
Classifying test sample 16/326
Prediction: 1
Classifying test sample 17/326
Prediction: 4
Classifying test sample 18/326
Prediction: 4
Classifying test sample 19/326
Prediction: 1
Classifying test sample 20/326
Prediction: 1
Classifying test sample 21/326
Prediction: 1
Classifying test sample 22/326
Prediction: 1
Classifying test s

In [14]:
import matplotlib.pyplot as plt
from scipy.fftpack import dct
from scipy import signal

# 設置中文字體支持
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False  # 解決負號顯示問題

# 視覺化 MFCC 特徵（不使用 numba/librosa）
if saved_samples:
    sample = saved_samples[0]  # 使用第一個樣本
    
    print(f"分析文件: {sample['filename']}")
    print(f"MFCC 特徵形狀: {sample['mfcc_features'].shape}")
    print()
    
    # 創建視覺化
    fig, axes = plt.subplots(4, 1, figsize=(14, 12))
    
    # 1. 原始音頻波形
    time_original = np.arange(len(sample['original_signal'])) / sample['rate']
    axes[0].plot(time_original, sample['original_signal'], linewidth=0.5)
    axes[0].set_title('原始音頻波形', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('時間 (秒)')
    axes[0].set_ylabel('振幅')
    axes[0].grid(True, alpha=0.3)
    
    # 2. 原始音頻的頻譜圖
    f, t, Sxx = signal.spectrogram(sample['original_signal'], sample['rate'])
    spec_display = axes[1].pcolormesh(t, f, 10 * np.log10(Sxx + 1e-10), shading='gouraud', cmap='viridis')
    axes[1].set_title('原始音頻頻譜圖', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('時間 (秒)')
    axes[1].set_ylabel('頻率 (Hz)')
    axes[1].set_ylim([0, 8000])  # 限制顯示範圍
    plt.colorbar(spec_display, ax=axes[1], label='功率 (dB)')
    
    # 3. MFCC 特徵視覺化 (熱力圖)
    mfcc_display = axes[2].imshow(sample['mfcc_features'].T, 
                                   aspect='auto', 
                                   origin='lower',
                                   cmap='viridis')
    axes[2].set_title('MFCC 特徵 (壓縮後的音頻表示)', fontsize=12, fontweight='bold')
    axes[2].set_xlabel('時間幀')
    axes[2].set_ylabel('MFCC 係數')
    plt.colorbar(mfcc_display, ax=axes[2], label='MFCC 值')
    
    # 4. 使用簡單的逆 DCT 嘗試近似重建 log-mel spectrogram
    try:
        # 對每個時間幀進行逆 DCT
        mfcc_data = sample['mfcc_features']
        n_mels = 40  # 使用的 mel bins 數量
        
        # 逆 DCT: MFCC -> log mel spectrogram (近似)
        log_mel_spec = dct(mfcc_data, type=3, axis=1, norm='ortho')[:, :n_mels]
        
        # 顯示重建的 log-mel spectrogram
        mel_display = axes[3].imshow(log_mel_spec.T,
                                     aspect='auto',
                                     origin='lower',
                                     cmap='viridis')
        axes[3].set_title('從 MFCC 重建的 Log-Mel 頻譜 (近似)', fontsize=12, fontweight='bold')
        axes[3].set_xlabel('時間幀')
        axes[3].set_ylabel('Mel 頻帶')
        plt.colorbar(mel_display, ax=axes[3], label='對數功率')
        
    except Exception as e:
        axes[3].text(0.5, 0.5, f'重建失敗: {str(e)}', 
                    ha='center', va='center', transform=axes[3].transAxes)
        axes[3].set_title('從 MFCC 重建的 Log-Mel 頻譜', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\n說明：")
    print("- 第1圖：原始音頻的時域波形")
    print("- 第2圖：原始音頻的頻譜圖（完整的頻率-時間表示）")
    print("- 第3圖：MFCC 特徵（壓縮到 13 個係數，2994 個時間幀）")
    print("- 第4圖：從 MFCC 逆 DCT 重建的 Log-Mel 頻譜（40 個 mel 頻帶）")
    print("\nMFCC 壓縮過程：")
    print("  音頻波形 → 短時傅立葉變換 → Mel 濾波器組 → 對數 → DCT → MFCC")
    print("\n遺失的信息：")
    print("  • 相位信息（無法從 MFCC 完美重建音頻）")
    print("  • 高頻細節（只保留前 13 個 DCT 係數）")
    print("  • Mel 頻帶以上的頻譜細節")
    print("\nMFCC 保留了音頻的主要頻譜包絡特徵，適合用於分類任務。")

NameError: name 'saved_samples' is not defined